In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from PIL import Image
import os


# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data_path = "/content/drive/MyDrive/class/캡스톤/베도승격/data/전처리_이미지"  # 본인의 데이터셋 경로로 변경

dataset = datasets.ImageFolder(root=data_path, transform=transform)

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# DataLoader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# ResNet
resnet_model = models.resnet18(pretrained=True)
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, len(dataset.classes))
resnet_model = resnet_model.to(device)

# loss_optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)

# train_model
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

# test_model
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    loss = running_loss / len(test_loader.dataset)
    accuracy = accuracy_score(all_labels, all_preds)
    return loss, accuracy

# 작동
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(resnet_model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = test(resnet_model, test_loader, criterion, device)
    print(f"Epoch {epoch + 1}/{num_epochs} -> Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# 데이터 경로 설정
data_path = "/content/drive/MyDrive/캡스톤_팀플/data/crop_img"

In [ ]:
# 이미지 크기 및 배치 크기 설정 (크기를 더 작게 조절)
img_size = (64, 64)
batch_size = 16

In [ ]:
# 데이터 불러오기 및 전처리
def load_data(data_path):
    images = []
    labels = []

    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                image_path = os.path.join(folder_path, filename)
                img = Image.open(image_path)
                img = img.resize(img_size)
                img_array = np.array(img) / 255.0  # Normalize
                images.append(img_array)
                labels.append(folder_name)

    return np.array(images), np.array(labels)

In [ ]:
# tqdm을 사용하여 데이터 로드
def load_data_with_progress(data_path):
    images = []
    labels = []

    for folder_name in tqdm(os.listdir(data_path), desc="Loading Data"):
        folder_path = os.path.join(data_path, folder_name)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                image_path = os.path.join(folder_path, filename)
                img = Image.open(image_path)
                img = img.resize(img_size)
                img_array = np.array(img) / 255.0  # Normalize
                images.append(img_array)
                labels.append(folder_name)

    return np.array(images), np.array(labels)

# 데이터 로드
images, labels = load_data_with_progress(data_path)

Loading Data: 100%|██████████| 80/80 [23:56<00:00, 17.95s/it]


In [ ]:
# 레이블 인코딩
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    images, labels_encoded, test_size=0.2, random_state=42
)

In [ ]:
# ResNet50 모델 불러오기
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(64, 64, 3))

In [ ]:
# 모델 구성
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation="relu"))  # 뉴런 수 증가
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(label_encoder.classes_), activation="softmax"))

In [ ]:
# 모델 컴파일
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),  # 학습률 조절
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
# 데이터 증강을 사용한 학습
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
train_datagen.fit(X_train)

In [ ]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [ ]:
# 모델 학습
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_test, y_test),
    epochs=100,  # 적절한 에폭 설정
    callbacks=[early_stopping]  # EarlyStopping 콜백 추가
)

Epoch 1/100
432/432 [==============================] - 63s 71ms/step - loss: 3.8054 - accuracy: 0.1474 - val_loss: 3.4247 - val_accuracy: 0.1876
Epoch 2/100
432/432 [==============================] - 29s 67ms/step - loss: 3.3211 - accuracy: 0.1868 - val_loss: 3.1595 - val_accuracy: 0.2218
Epoch 3/100
432/432 [==============================] - 29s 67ms/step - loss: 3.1654 - accuracy: 0.2103 - val_loss: 3.0588 - val_accuracy: 0.2397
Epoch 4/100
432/432 [==============================] - 29s 68ms/step - loss: 3.0907 - accuracy: 0.2163 - val_loss: 3.1200 - val_accuracy: 0.2171
Epoch 5/100
432/432 [==============================] - 29s 67ms/step - loss: 3.0411 - accuracy: 0.2282 - val_loss: 2.9048 - val_accuracy: 0.2467
Epoch 6/100
432/432 [==============================] - 30s 68ms/step - loss: 2.9809 - accuracy: 0.2430 - val_loss: 2.8938 - val_accuracy: 0.2490
Epoch 7/100
432/432 [==============================] - 30s 68ms/step - loss: 2.9299 - accuracy: 0.2443 - val_loss: 2.8804 - val_ac